In [1]:
import csv
import numpy as np 
import pandas as pd
from datetime import datetime as dt
import pickle

pd.set_option('display.max_rows', 4000)
pd.set_option('display.max_columns', 200)

**Get all user IDs**

all user ids, stored in 'userID_dict' in the format of:

userID_dict[user_id] = username

In [2]:
temp1D = pd.read_csv("dict_1d_usernames.csv", low_memory=False)
temp2D = pd.read_csv("dict_2d_usernames.csv", low_memory=False)
# temp1D.head()
# temp2D.head()

In [3]:
userID_dict = {}

for i in range(len(temp1D)):
    userID_dict[temp1D['user_id'][i]] = temp1D['username'][i]

for i in range(len(temp2D)):
    userID_dict[temp2D['user_id'][i]] = temp2D['username'][i]

len(userID_dict)

5001

**Read in IRA tweet set**

In [2]:
iraDF = pd.read_csv('ira_tweets_csv_unhashed.csv', low_memory=False)

In [3]:
unique_list = iraDF.user_display_name.unique()

In [4]:
len(unique_list)

3443

In [5]:
unique_list2 = iraDF.userid.unique()

In [6]:
len(unique_list2)

3479

In [5]:
print(len(iraDF))

8768633


In [6]:
iraDF.tail(5)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,in_reply_to_tweetid,in_reply_to_userid,quoted_tweet_tweetid,is_retweet,retweet_userid,retweet_tweetid,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices
8768628,552931773240209408,2231059909,Невские Новости,NevnovPR,Санкт-Петербург,ИА Невские новости по районам. Самые свежие но...,http://t.co/vdTR6Tj1Iq,4693,2287,2013-12-05,ru,ru,На КАД автомобиль влетел в ограждение: пассажи...,2015-01-07 20:56,twitterfeed,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,[СПб],[http://bit.ly/1xGnfB8],NaN,NaN
8768629,430175706794835968,2231059909,Невские Новости,NevnovPR,Санкт-Петербург,ИА Невские новости по районам. Самые свежие но...,http://t.co/vdTR6Tj1Iq,4693,2287,2013-12-05,ru,ru,Финал Кубка Содружества: украинцы разгромили с...,2014-02-03 03:07,twitterfeed,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,[СПб],NaN,NaN,NaN
8768630,488096513516523521,2231059909,Невские Новости,NevnovPR,Санкт-Петербург,ИА Невские новости по районам. Самые свежие но...,http://t.co/vdTR6Tj1Iq,4693,2287,2013-12-05,ru,ru,ГИБДД: Водитель фуры серьезно пострадал при па...,2014-07-12 23:04,twitterfeed,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,[СПб],NaN,NaN,NaN
8768631,559036918319816704,1949085066,kpesmina,kpesmina,Манила,Люблю гулять у воды. Пишу на когда как. Не люб...,http://t.co/8TrkgDfbEy,558,453,2013-10-09,ru,ru,RT @berdnikov: Обстрел Донецка и Луганска - эт...,2015-01-24 17:16,meceslav,NaN,NaN,NaN,True,17964137.0,5.590306e+17,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,[17964137],NaN
8768632,518502331537965056,1949085066,kpesmina,kpesmina,Манила,Люблю гулять у воды. Пишу на когда как. Не люб...,http://t.co/8TrkgDfbEy,558,453,2013-10-09,ru,ru,"RT @LevSharansky: Мы, украинцы, изначально зна...",2014-10-04 20:46,newtwittersky,NaN,NaN,NaN,True,253261225.0,5.184983e+17,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,[253261225],NaN


**Create DataFrame for IRA -> User Contact Points**

In [11]:
contactPointsDF = pd.DataFrame(columns=['username','user_id','ira_username','ira_user_id',
                                       'tweet', 'tweet_date', 'meantions_list'])

**Loop through IRA data set, checking for contact with our current user set, if contact was established, add to contactPointsDF**

In [12]:
total = str(len(iraDF))
count = 1
#loop through ira tweet DF
for tweet in range(len(iraDF)):
    mention_string = str(iraDF['user_mentions'][tweet])
    #if mention list exists, save and reformat as it is a string not a list
    if mention_string != "[]" and mention_string != 'nan':  
        mention_list = mention_string.split(', ')
        mention_list[0] = mention_list[0][1:]
        mention_list[-1] = mention_list[-1][:-1]
        for x in range(len(mention_list)):
            mention_list[x] = int(mention_list[x])
#         for j in range(len(mention_list)):
#             mention_list[j] = int(mention_list[j])

        #check against user ids in our userID_dict
        for mention in mention_list:
            temp_key = mention
            if temp_key in userID_dict.keys():
#                 print("found!")
                #if a user id was mentioned, add to contactPointsDF
                contactPointsDF = contactPointsDF.append(
                    {'username' : userID_dict[temp_key],
                      'user_id' : temp_key,
                      'ira_username' : iraDF['user_screen_name'][tweet],
                      'ira_user_id' : iraDF['userid'][tweet],
                      'tweet' : iraDF['tweet_text'][tweet],
                      'tweet_date' : iraDF['tweet_time'][tweet],
                      'meantions_list' : mention_list,
                     }, ignore_index=True)
    if count % 1000 == 0:
        print(str(count) + " / " + total)
    count += 1

1000 / 8768633
2000 / 8768633
3000 / 8768633
4000 / 8768633
5000 / 8768633
6000 / 8768633
7000 / 8768633
8000 / 8768633
9000 / 8768633
10000 / 8768633
11000 / 8768633
12000 / 8768633
13000 / 8768633
14000 / 8768633
15000 / 8768633
16000 / 8768633
17000 / 8768633
18000 / 8768633
19000 / 8768633
20000 / 8768633
21000 / 8768633
22000 / 8768633
23000 / 8768633
24000 / 8768633
25000 / 8768633
26000 / 8768633
27000 / 8768633
28000 / 8768633
29000 / 8768633
30000 / 8768633
31000 / 8768633
32000 / 8768633
33000 / 8768633
34000 / 8768633
35000 / 8768633
36000 / 8768633
37000 / 8768633
38000 / 8768633
39000 / 8768633
40000 / 8768633
41000 / 8768633
42000 / 8768633
43000 / 8768633
44000 / 8768633
45000 / 8768633
46000 / 8768633
47000 / 8768633
48000 / 8768633
49000 / 8768633
50000 / 8768633
51000 / 8768633
52000 / 8768633
53000 / 8768633
54000 / 8768633
55000 / 8768633
56000 / 8768633
57000 / 8768633
58000 / 8768633
59000 / 8768633
60000 / 8768633
61000 / 8768633
62000 / 8768633
63000 / 8768633
6

490000 / 8768633
491000 / 8768633
492000 / 8768633
493000 / 8768633
494000 / 8768633
495000 / 8768633
496000 / 8768633
497000 / 8768633
498000 / 8768633
499000 / 8768633
500000 / 8768633
501000 / 8768633
502000 / 8768633
503000 / 8768633
504000 / 8768633
505000 / 8768633
506000 / 8768633
507000 / 8768633
508000 / 8768633
509000 / 8768633
510000 / 8768633
511000 / 8768633
512000 / 8768633
513000 / 8768633
514000 / 8768633
515000 / 8768633
516000 / 8768633
517000 / 8768633
518000 / 8768633
519000 / 8768633
520000 / 8768633
521000 / 8768633
522000 / 8768633
523000 / 8768633
524000 / 8768633
525000 / 8768633
526000 / 8768633
527000 / 8768633
528000 / 8768633
529000 / 8768633
530000 / 8768633
531000 / 8768633
532000 / 8768633
533000 / 8768633
534000 / 8768633
535000 / 8768633
536000 / 8768633
537000 / 8768633
538000 / 8768633
539000 / 8768633
540000 / 8768633
541000 / 8768633
542000 / 8768633
543000 / 8768633
544000 / 8768633
545000 / 8768633
546000 / 8768633
547000 / 8768633
548000 / 87686

972000 / 8768633
973000 / 8768633
974000 / 8768633
975000 / 8768633
976000 / 8768633
977000 / 8768633
978000 / 8768633
979000 / 8768633
980000 / 8768633
981000 / 8768633
982000 / 8768633
983000 / 8768633
984000 / 8768633
985000 / 8768633
986000 / 8768633
987000 / 8768633
988000 / 8768633
989000 / 8768633
990000 / 8768633
991000 / 8768633
992000 / 8768633
993000 / 8768633
994000 / 8768633
995000 / 8768633
996000 / 8768633
997000 / 8768633
998000 / 8768633
999000 / 8768633
1000000 / 8768633
1001000 / 8768633
1002000 / 8768633
1003000 / 8768633
1004000 / 8768633
1005000 / 8768633
1006000 / 8768633
1007000 / 8768633
1008000 / 8768633
1009000 / 8768633
1010000 / 8768633
1011000 / 8768633
1012000 / 8768633
1013000 / 8768633
1014000 / 8768633
1015000 / 8768633
1016000 / 8768633
1017000 / 8768633
1018000 / 8768633
1019000 / 8768633
1020000 / 8768633
1021000 / 8768633
1022000 / 8768633
1023000 / 8768633
1024000 / 8768633
1025000 / 8768633
1026000 / 8768633
1027000 / 8768633
1028000 / 8768633
10

1429000 / 8768633
1430000 / 8768633
1431000 / 8768633
1432000 / 8768633
1433000 / 8768633
1434000 / 8768633
1435000 / 8768633
1436000 / 8768633
1437000 / 8768633
1438000 / 8768633
1439000 / 8768633
1440000 / 8768633
1441000 / 8768633
1442000 / 8768633
1443000 / 8768633
1444000 / 8768633
1445000 / 8768633
1446000 / 8768633
1447000 / 8768633
1448000 / 8768633
1449000 / 8768633
1450000 / 8768633
1451000 / 8768633
1452000 / 8768633
1453000 / 8768633
1454000 / 8768633
1455000 / 8768633
1456000 / 8768633
1457000 / 8768633
1458000 / 8768633
1459000 / 8768633
1460000 / 8768633
1461000 / 8768633
1462000 / 8768633
1463000 / 8768633
1464000 / 8768633
1465000 / 8768633
1466000 / 8768633
1467000 / 8768633
1468000 / 8768633
1469000 / 8768633
1470000 / 8768633
1471000 / 8768633
1472000 / 8768633
1473000 / 8768633
1474000 / 8768633
1475000 / 8768633
1476000 / 8768633
1477000 / 8768633
1478000 / 8768633
1479000 / 8768633
1480000 / 8768633
1481000 / 8768633
1482000 / 8768633
1483000 / 8768633
1484000 / 

1885000 / 8768633
1886000 / 8768633
1887000 / 8768633
1888000 / 8768633
1889000 / 8768633
1890000 / 8768633
1891000 / 8768633
1892000 / 8768633
1893000 / 8768633
1894000 / 8768633
1895000 / 8768633
1896000 / 8768633
1897000 / 8768633
1898000 / 8768633
1899000 / 8768633
1900000 / 8768633
1901000 / 8768633
1902000 / 8768633
1903000 / 8768633
1904000 / 8768633
1905000 / 8768633
1906000 / 8768633
1907000 / 8768633
1908000 / 8768633
1909000 / 8768633
1910000 / 8768633
1911000 / 8768633
1912000 / 8768633
1913000 / 8768633
1914000 / 8768633
1915000 / 8768633
1916000 / 8768633
1917000 / 8768633
1918000 / 8768633
1919000 / 8768633
1920000 / 8768633
1921000 / 8768633
1922000 / 8768633
1923000 / 8768633
1924000 / 8768633
1925000 / 8768633
1926000 / 8768633
1927000 / 8768633
1928000 / 8768633
1929000 / 8768633
1930000 / 8768633
1931000 / 8768633
1932000 / 8768633
1933000 / 8768633
1934000 / 8768633
1935000 / 8768633
1936000 / 8768633
1937000 / 8768633
1938000 / 8768633
1939000 / 8768633
1940000 / 

...

6901000 / 8768633
6902000 / 8768633
6903000 / 8768633
6904000 / 8768633
6905000 / 8768633
6906000 / 8768633
6907000 / 8768633
6908000 / 8768633
6909000 / 8768633
6910000 / 8768633
6911000 / 8768633
6912000 / 8768633
6913000 / 8768633
6914000 / 8768633
6915000 / 8768633
6916000 / 8768633
6917000 / 8768633
6918000 / 8768633
6919000 / 8768633
6920000 / 8768633
6921000 / 8768633
6922000 / 8768633
6923000 / 8768633
6924000 / 8768633
6925000 / 8768633
6926000 / 8768633
6927000 / 8768633
6928000 / 8768633
6929000 / 8768633
6930000 / 8768633
6931000 / 8768633
6932000 / 8768633
6933000 / 8768633
6934000 / 8768633
6935000 / 8768633
6936000 / 8768633
6937000 / 8768633
6938000 / 8768633
6939000 / 8768633
6940000 / 8768633
6941000 / 8768633
6942000 / 8768633
6943000 / 8768633
6944000 / 8768633
6945000 / 8768633
6946000 / 8768633
6947000 / 8768633
6948000 / 8768633
6949000 / 8768633
6950000 / 8768633
6951000 / 8768633
6952000 / 8768633
6953000 / 8768633
6954000 / 8768633
6955000 / 8768633
6956000 / 

7357000 / 8768633
7358000 / 8768633
7359000 / 8768633
7360000 / 8768633
7361000 / 8768633
7362000 / 8768633
7363000 / 8768633
7364000 / 8768633
7365000 / 8768633
7366000 / 8768633
7367000 / 8768633
7368000 / 8768633
7369000 / 8768633
7370000 / 8768633
7371000 / 8768633
7372000 / 8768633
7373000 / 8768633
7374000 / 8768633
7375000 / 8768633
7376000 / 8768633
7377000 / 8768633
7378000 / 8768633
7379000 / 8768633
7380000 / 8768633
7381000 / 8768633
7382000 / 8768633
7383000 / 8768633
7384000 / 8768633
7385000 / 8768633
7386000 / 8768633
7387000 / 8768633
7388000 / 8768633
7389000 / 8768633
7390000 / 8768633
7391000 / 8768633
7392000 / 8768633
7393000 / 8768633
7394000 / 8768633
7395000 / 8768633
7396000 / 8768633
7397000 / 8768633
7398000 / 8768633
7399000 / 8768633
7400000 / 8768633
7401000 / 8768633
7402000 / 8768633
7403000 / 8768633
7404000 / 8768633
7405000 / 8768633
7406000 / 8768633
7407000 / 8768633
7408000 / 8768633
7409000 / 8768633
7410000 / 8768633
7411000 / 8768633
7412000 / 

7813000 / 8768633
7814000 / 8768633
7815000 / 8768633
7816000 / 8768633
7817000 / 8768633
7818000 / 8768633
7819000 / 8768633
7820000 / 8768633
7821000 / 8768633
7822000 / 8768633
7823000 / 8768633
7824000 / 8768633
7825000 / 8768633
7826000 / 8768633
7827000 / 8768633
7828000 / 8768633
7829000 / 8768633
7830000 / 8768633
7831000 / 8768633
7832000 / 8768633
7833000 / 8768633
7834000 / 8768633
7835000 / 8768633
7836000 / 8768633
7837000 / 8768633
7838000 / 8768633
7839000 / 8768633
7840000 / 8768633
7841000 / 8768633
7842000 / 8768633
7843000 / 8768633
7844000 / 8768633
7845000 / 8768633
7846000 / 8768633
7847000 / 8768633
7848000 / 8768633
7849000 / 8768633
7850000 / 8768633
7851000 / 8768633
7852000 / 8768633
7853000 / 8768633
7854000 / 8768633
7855000 / 8768633
7856000 / 8768633
7857000 / 8768633
7858000 / 8768633
7859000 / 8768633
7860000 / 8768633
7861000 / 8768633
7862000 / 8768633
7863000 / 8768633
7864000 / 8768633
7865000 / 8768633
7866000 / 8768633
7867000 / 8768633
7868000 / 

8269000 / 8768633
8270000 / 8768633
8271000 / 8768633
8272000 / 8768633
8273000 / 8768633
8274000 / 8768633
8275000 / 8768633
8276000 / 8768633
8277000 / 8768633
8278000 / 8768633
8279000 / 8768633
8280000 / 8768633
8281000 / 8768633
8282000 / 8768633
8283000 / 8768633
8284000 / 8768633
8285000 / 8768633
8286000 / 8768633
8287000 / 8768633
8288000 / 8768633
8289000 / 8768633
8290000 / 8768633
8291000 / 8768633
8292000 / 8768633
8293000 / 8768633
8294000 / 8768633
8295000 / 8768633
8296000 / 8768633
8297000 / 8768633
8298000 / 8768633
8299000 / 8768633
8300000 / 8768633
8301000 / 8768633
8302000 / 8768633
8303000 / 8768633
8304000 / 8768633
8305000 / 8768633
8306000 / 8768633
8307000 / 8768633
8308000 / 8768633
8309000 / 8768633
8310000 / 8768633
8311000 / 8768633
8312000 / 8768633
8313000 / 8768633
8314000 / 8768633
8315000 / 8768633
8316000 / 8768633
8317000 / 8768633
8318000 / 8768633
8319000 / 8768633
8320000 / 8768633
8321000 / 8768633
8322000 / 8768633
8323000 / 8768633
8324000 / 

8725000 / 8768633
8726000 / 8768633
8727000 / 8768633
8728000 / 8768633
8729000 / 8768633
8730000 / 8768633
8731000 / 8768633
8732000 / 8768633
8733000 / 8768633
8734000 / 8768633
8735000 / 8768633
8736000 / 8768633
8737000 / 8768633
8738000 / 8768633
8739000 / 8768633
8740000 / 8768633
8741000 / 8768633
8742000 / 8768633
8743000 / 8768633
8744000 / 8768633
8745000 / 8768633
8746000 / 8768633
8747000 / 8768633
8748000 / 8768633
8749000 / 8768633
8750000 / 8768633
8751000 / 8768633
8752000 / 8768633
8753000 / 8768633
8754000 / 8768633
8755000 / 8768633
8756000 / 8768633
8757000 / 8768633
8758000 / 8768633
8759000 / 8768633
8760000 / 8768633
8761000 / 8768633
8762000 / 8768633
8763000 / 8768633
8764000 / 8768633
8765000 / 8768633
8766000 / 8768633
8767000 / 8768633
8768000 / 8768633


**Now create a final DF which is more user-centric, i.e., a row for each user as opposed to a row for each tweet**


In [13]:
contactedUsers = contactPointsDF.username.unique()

In [14]:
len(contactPointsDF)

187367

In [16]:
contactPointsDF.to_csv('contact_IRAtweets.csv')

In [17]:
len(contactedUsers)

4996

In [22]:
contactDF = pd.DataFrame(columns=['username','user_id','ira_username','ira_user_id',
                                       'tweet', 'tweet_date', 'mentions_list'])

In [23]:
def get_key(val): 
    for key, value in userID_dict.items(): 
        if val == value: 
            return key

In [25]:
print("hello")

hello


In [ ]:
size = str(len(contactedUsers))
count = 1
for user in contactedUsers:
    ira_username_temp = []
    ira_userid_temp = []
    tweet_temp = []
    tweet_date_temp = []
    mentions_list_temp = []
    for tweet in range(len(contactPointsDF)):
        if(contactPointsDF['username'][tweet] == user):
            ira_username_temp.append(contactPointsDF['ira_username'][tweet])
            ira_userid_temp.append(contactPointsDF['ira_user_id'][tweet])
            tweet_temp.append(contactPointsDF['tweet'][tweet])
            tweet_date_temp.append(contactPointsDF['tweet_date'][tweet])
            mentions_list_temp.append(contactPointsDF['meantions_list'][tweet])
    contactDF = contactDF.append(
                    {'username' : user,
                      'user_id' : get_key(user),
                      'ira_username' : ira_username_temp,
                      'ira_user_id' : ira_userid_temp,
                      'tweet' : tweet_temp,
                      'tweet_date' : tweet_date_temp,
                      'mentions_list' : mentions_list_temp,
                     }, ignore_index=True)
    print(str(count) + " / " + size)
    count += 1

1 / 4996
2 / 4996
3 / 4996
4 / 4996
5 / 4996
6 / 4996
7 / 4996
8 / 4996
9 / 4996
10 / 4996
11 / 4996
12 / 4996
13 / 4996
14 / 4996
15 / 4996
16 / 4996
17 / 4996
18 / 4996
19 / 4996
20 / 4996
21 / 4996
22 / 4996
23 / 4996
24 / 4996
25 / 4996
26 / 4996
27 / 4996
28 / 4996
29 / 4996
30 / 4996
31 / 4996
32 / 4996
33 / 4996
34 / 4996
35 / 4996
36 / 4996
37 / 4996
38 / 4996
39 / 4996
40 / 4996
41 / 4996
42 / 4996
43 / 4996
44 / 4996
45 / 4996
46 / 4996
47 / 4996
48 / 4996
49 / 4996
50 / 4996
51 / 4996
52 / 4996
53 / 4996
54 / 4996
55 / 4996
56 / 4996
57 / 4996
58 / 4996
59 / 4996
60 / 4996
61 / 4996
62 / 4996
63 / 4996
64 / 4996
65 / 4996
66 / 4996
67 / 4996
68 / 4996
69 / 4996
70 / 4996
71 / 4996
72 / 4996
73 / 4996
74 / 4996
75 / 4996
76 / 4996
77 / 4996
78 / 4996
79 / 4996
80 / 4996
81 / 4996
82 / 4996
83 / 4996
84 / 4996
85 / 4996
86 / 4996
87 / 4996
88 / 4996
89 / 4996
90 / 4996
91 / 4996
92 / 4996
93 / 4996
94 / 4996
95 / 4996
96 / 4996
97 / 4996
98 / 4996
99 / 4996
100 / 4996
101 / 49

756 / 4996
757 / 4996
758 / 4996
759 / 4996
760 / 4996
761 / 4996
762 / 4996
763 / 4996
764 / 4996
765 / 4996
766 / 4996
767 / 4996
768 / 4996
769 / 4996
770 / 4996
771 / 4996
772 / 4996
773 / 4996
774 / 4996
775 / 4996
776 / 4996
777 / 4996
778 / 4996
779 / 4996
780 / 4996
781 / 4996
782 / 4996
783 / 4996
784 / 4996
785 / 4996
786 / 4996
787 / 4996
788 / 4996
789 / 4996
790 / 4996
791 / 4996
792 / 4996
793 / 4996
794 / 4996
795 / 4996
796 / 4996
797 / 4996
798 / 4996
799 / 4996
800 / 4996
801 / 4996
802 / 4996
803 / 4996
804 / 4996
805 / 4996
806 / 4996
807 / 4996
808 / 4996
809 / 4996
810 / 4996
811 / 4996
812 / 4996
813 / 4996
814 / 4996
815 / 4996
816 / 4996
817 / 4996
818 / 4996
819 / 4996
820 / 4996
821 / 4996
822 / 4996
823 / 4996
824 / 4996
825 / 4996
826 / 4996
827 / 4996
828 / 4996
829 / 4996
830 / 4996
831 / 4996
832 / 4996
833 / 4996
834 / 4996
835 / 4996
836 / 4996
837 / 4996
838 / 4996
839 / 4996
840 / 4996
841 / 4996
842 / 4996
843 / 4996
844 / 4996
845 / 4996
846 / 4996

1460 / 4996
1461 / 4996
1462 / 4996
1463 / 4996
1464 / 4996
1465 / 4996
1466 / 4996
1467 / 4996
1468 / 4996
1469 / 4996
1470 / 4996
1471 / 4996
1472 / 4996
1473 / 4996
1474 / 4996
1475 / 4996
1476 / 4996
1477 / 4996
1478 / 4996
1479 / 4996
1480 / 4996
1481 / 4996
1482 / 4996
1483 / 4996
1484 / 4996
1485 / 4996
1486 / 4996
1487 / 4996
1488 / 4996
1489 / 4996
1490 / 4996
1491 / 4996
1492 / 4996
1493 / 4996
1494 / 4996
1495 / 4996
1496 / 4996
1497 / 4996
1498 / 4996
1499 / 4996
1500 / 4996
1501 / 4996
1502 / 4996
1503 / 4996
1504 / 4996
1505 / 4996
1506 / 4996
1507 / 4996
1508 / 4996
1509 / 4996
1510 / 4996
1511 / 4996
1512 / 4996
1513 / 4996
1514 / 4996
1515 / 4996
1516 / 4996
1517 / 4996
1518 / 4996
1519 / 4996
1520 / 4996
1521 / 4996
1522 / 4996
1523 / 4996
1524 / 4996
1525 / 4996
1526 / 4996
1527 / 4996
1528 / 4996
1529 / 4996
1530 / 4996
1531 / 4996
1532 / 4996
1533 / 4996
1534 / 4996
1535 / 4996
1536 / 4996
1537 / 4996
1538 / 4996
1539 / 4996
1540 / 4996
1541 / 4996
1542 / 4996
1543

2143 / 4996
2144 / 4996
2145 / 4996
2146 / 4996
2147 / 4996
2148 / 4996
2149 / 4996
2150 / 4996
2151 / 4996
2152 / 4996
2153 / 4996
2154 / 4996
2155 / 4996
2156 / 4996
2157 / 4996
2158 / 4996
2159 / 4996
2160 / 4996
2161 / 4996
2162 / 4996
2163 / 4996
2164 / 4996
2165 / 4996
2166 / 4996
2167 / 4996
2168 / 4996
2169 / 4996
2170 / 4996
2171 / 4996
2172 / 4996
2173 / 4996
2174 / 4996
2175 / 4996
2176 / 4996
2177 / 4996
2178 / 4996
2179 / 4996
2180 / 4996
2181 / 4996
2182 / 4996
2183 / 4996
2184 / 4996
2185 / 4996
2186 / 4996
2187 / 4996
2188 / 4996
2189 / 4996
2190 / 4996
2191 / 4996
2192 / 4996
2193 / 4996
2194 / 4996
2195 / 4996
2196 / 4996
2197 / 4996
2198 / 4996
2199 / 4996
2200 / 4996
2201 / 4996
2202 / 4996
2203 / 4996
2204 / 4996
2205 / 4996
2206 / 4996
2207 / 4996
2208 / 4996
2209 / 4996
2210 / 4996
2211 / 4996
2212 / 4996
2213 / 4996
2214 / 4996
2215 / 4996
2216 / 4996
2217 / 4996
2218 / 4996
2219 / 4996
2220 / 4996
2221 / 4996
2222 / 4996
2223 / 4996
2224 / 4996
2225 / 4996
2226

2826 / 4996
2827 / 4996
2828 / 4996
2829 / 4996
2830 / 4996
2831 / 4996
2832 / 4996
2833 / 4996
2834 / 4996
2835 / 4996
2836 / 4996
2837 / 4996
2838 / 4996
2839 / 4996
2840 / 4996
2841 / 4996
2842 / 4996
2843 / 4996
2844 / 4996
2845 / 4996
2846 / 4996
2847 / 4996
2848 / 4996
2849 / 4996
2850 / 4996
2851 / 4996
2852 / 4996
2853 / 4996
2854 / 4996
2855 / 4996
2856 / 4996
2857 / 4996
2858 / 4996
2859 / 4996
2860 / 4996
2861 / 4996
2862 / 4996
2863 / 4996
2864 / 4996
2865 / 4996
2866 / 4996
2867 / 4996
2868 / 4996
2869 / 4996
2870 / 4996
2871 / 4996
2872 / 4996
2873 / 4996
2874 / 4996
2875 / 4996
2876 / 4996
2877 / 4996
2878 / 4996
2879 / 4996
2880 / 4996
2881 / 4996
2882 / 4996
2883 / 4996
2884 / 4996
2885 / 4996
2886 / 4996
2887 / 4996
2888 / 4996
2889 / 4996
2890 / 4996
2891 / 4996
2892 / 4996
2893 / 4996
2894 / 4996
2895 / 4996
2896 / 4996
2897 / 4996
2898 / 4996
2899 / 4996
2900 / 4996
2901 / 4996
2902 / 4996
2903 / 4996
2904 / 4996
2905 / 4996
2906 / 4996
2907 / 4996
2908 / 4996
2909

3509 / 4996
3510 / 4996
3511 / 4996
3512 / 4996
3513 / 4996
3514 / 4996
3515 / 4996
3516 / 4996
3517 / 4996
3518 / 4996
3519 / 4996
3520 / 4996
3521 / 4996
3522 / 4996
3523 / 4996
3524 / 4996
3525 / 4996
3526 / 4996
3527 / 4996
3528 / 4996
3529 / 4996
3530 / 4996
3531 / 4996
3532 / 4996
3533 / 4996
3534 / 4996
3535 / 4996
3536 / 4996
3537 / 4996
3538 / 4996
3539 / 4996
3540 / 4996
3541 / 4996
3542 / 4996
3543 / 4996
3544 / 4996
3545 / 4996
3546 / 4996
3547 / 4996
3548 / 4996
3549 / 4996
3550 / 4996
3551 / 4996
3552 / 4996
3553 / 4996
3554 / 4996
3555 / 4996
3556 / 4996
3557 / 4996
3558 / 4996
3559 / 4996
3560 / 4996
3561 / 4996
3562 / 4996
3563 / 4996
3564 / 4996
3565 / 4996
3566 / 4996
3567 / 4996
3568 / 4996
3569 / 4996
3570 / 4996
3571 / 4996
3572 / 4996
3573 / 4996
3574 / 4996
3575 / 4996
3576 / 4996
3577 / 4996
3578 / 4996
3579 / 4996
3580 / 4996
3581 / 4996
3582 / 4996
3583 / 4996
3584 / 4996
3585 / 4996
3586 / 4996
3587 / 4996
3588 / 4996
3589 / 4996
3590 / 4996
3591 / 4996
3592

In [ ]:
contactDF.to_csv('contact_IRAtoUser.csv')

In [28]:
print(len(contactDF))

7261


In [29]:
len(contactDF.username.unique())

4996

In [30]:
contactDF.head()

,username,user_id,ira_username,ira_user_id,tweet,tweet_date,mentions_list
0,Borisovalustnah,2786446638,"[Filatovalechosl, Filatovalechosl, Filatovalec...","[2785702231, 2785702231, 2785702231, 279001131...",[RT @Borisovalustnah: 2d графические движки на...,"[2014-09-10 08:16, 2015-01-26 15:09, 2015-03-2...","[[2786446638], [2786446638], [2786446638], [27..."
1,manniefeathers6,763152148846653440,"[CovfefeNationUS, CovfefeNationUS, CovfefeNati...","[743166519157227520, 743166519157227520, 74316...",[RT @manniefeathers6: @tfgnews @POTUSADJT He E...,"[2017-01-20 18:44, 2016-12-12 00:16, 2017-03-0...","[[763152148846653440, 3435613942, 743166519157..."
2,Lenba_Spb,511617963,"[SashaGreece, Irina_Goooor, RomaBulavin, delfa...","[2433227806, 2496142952, 1647547494, 243305898...",[RT @Lenba_Spb: Toyota готовит новый кроссовер...,"[2015-02-01 21:48, 2015-02-11 07:58, 2014-12-1...","[[511617963], [511617963], [511617963], [51161..."
3,melaniesgore,15931176,"[CovfefeNationUS, CovfefeNationUS, CovfefeNati...","[743166519157227520, 743166519157227520, 74316...",[RT @melaniesgore: What would #Hillary support...,"[2016-12-06 08:23, 2017-01-20 05:13, 2016-12-0...","[[15931176], [15931176], [15931176, 2828427017..."
4,papilosha,2457294401,"[ArturBuranof, BogdanKravcov, RogovIlia, AlexH...","[2233297178, 2422793627, 2542876557, 258650658...",[RT @papilosha: Ой!От всего сердца!Главное-здо...,"[2016-12-17 05:58, 2017-01-15 19:18, 2016-10-1...","[[2457294401], [2457294401], [2457294401, 1240..."


In [31]:
for i in range(len(contactDF)):
    if contactDF['username'][i] == 'Borisovalustnah':
        print(str(i))

0
2265


In [64]:
print(str(len(contactDF) - 2265))

4996


In [65]:
newContactDF = contactDF[2265:]

In [66]:
len(newContactDF)

4996

In [67]:
newContactDF.tail()

,username,user_id,ira_username,ira_user_id,tweet,tweet_date,mentions_list
7256,den_lsko,467714192,"[MaxDementiev, coldwar20_ru]","[2808833544, 2351739462]","[@den_lsko @kononenkome жить вообще вредно, RT...","[2015-03-30 16:31, 2015-03-01 14:55]","[[467714192, 14256540], [467714192]]"
7257,_givethemhell,1672955719,"[Iakovlevaagambr, MaxDementiev]","[2787255740, 2808833544]",[RT @_givethemhell: мама будет спрашивать че э...,"[2014-09-17 19:48, 2014-10-28 08:22]","[[1672955719], [1672955719]]"
7258,ghost_of_a_deer,2605509277,"[MaryMozhaiskaya, MaxDementiev]","[2721995401, 2808833544]","[@DashaVad #оцениваюоформу 👌 - в норме, RT @D...","[2016-02-27 19:12, 2015-03-20 16:50]","[[2605509277], [2605509277, 2320149711, 228489..."
7259,Renathy777,2584562131,"[CovfefeNationUS, CovfefeNationUS]","[743166519157227520, 743166519157227520]","[@Renathy777 https://t.co/Gpke5MgQjM, RT @Rena...","[2017-03-01 04:22, 2017-03-01 04:22]","[[2584562131], [2584562131, 2746979823]]"
7260,RIPacheco79,1144714976,"[CovfefeNationUS, CovfefeNationUS]","[743166519157227520, 743166519157227520]",[@RIPacheco79 @SlicksTweetz @united Let's get ...,"[2017-04-11 04:40, 2017-04-11 04:39]","[[1144714976, 78431908, 260907612], [114471497..."


In [68]:
newContactDF= newContactDF.reset_index()

In [69]:
newContactDF = newContactDF.drop(['index'], axis=1)
newContactDF.head()

,username,user_id,ira_username,ira_user_id,tweet,tweet_date,mentions_list
0,Borisovalustnah,2786446638,"[Filatovalechosl, Filatovalechosl, Filatovalec...","[2785702231, 2785702231, 2785702231, 279001131...",[RT @Borisovalustnah: 2d графические движки на...,"[2014-09-10 08:16, 2015-01-26 15:09, 2015-03-2...","[[2786446638], [2786446638], [2786446638], [27..."
1,manniefeathers6,763152148846653440,"[CovfefeNationUS, CovfefeNationUS, CovfefeNati...","[743166519157227520, 743166519157227520, 74316...",[RT @manniefeathers6: @tfgnews @POTUSADJT He E...,"[2017-01-20 18:44, 2016-12-12 00:16, 2017-03-0...","[[763152148846653440, 3435613942, 743166519157..."
2,Lenba_Spb,511617963,"[SashaGreece, Irina_Goooor, RomaBulavin, delfa...","[2433227806, 2496142952, 1647547494, 243305898...",[RT @Lenba_Spb: Toyota готовит новый кроссовер...,"[2015-02-01 21:48, 2015-02-11 07:58, 2014-12-1...","[[511617963], [511617963], [511617963], [51161..."
3,melaniesgore,15931176,"[CovfefeNationUS, CovfefeNationUS, CovfefeNati...","[743166519157227520, 743166519157227520, 74316...",[RT @melaniesgore: What would #Hillary support...,"[2016-12-06 08:23, 2017-01-20 05:13, 2016-12-0...","[[15931176], [15931176], [15931176, 2828427017..."
4,papilosha,2457294401,"[ArturBuranof, BogdanKravcov, RogovIlia, AlexH...","[2233297178, 2422793627, 2542876557, 258650658...",[RT @papilosha: Ой!От всего сердца!Главное-здо...,"[2016-12-17 05:58, 2017-01-15 19:18, 2016-10-1...","[[2457294401], [2457294401], [2457294401, 1240..."


In [71]:
# newContactDF.to_csv('contact_IRAtoUser.csv')

In [72]:
# testing = newContactDF['mentions_list'][1]

In [73]:
# len(testing)

In [74]:
# tempdf = newContactDF
# tempdf['new'] = 0
# tempdf.head()

In [75]:
newContactDF['contact_points'] = 0
# print(testing[0][0])
for h in range(len(newContactDF)):
# for h in range(100):
    testing = newContactDF['mentions_list'][h]
    temp = []
    for k in range(len(testing)):
        for l in range(len(testing[k])):
            temp.append(testing[k][l])
    newContactDF['mentions_list'][h] = temp
    newContactDF['contact_points'][h] = len(temp)
#     print(temp)

/home/rhetthanscom/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/rhetthanscom/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [85]:
newContactDF[100:200]

,username,user_id,ira_username,ira_user_id,tweet,tweet_date,mentions_list,contact_points
100,BellDavidC,404468324,"[WorldOfHashtags, ChrixMorgan, EdmundRJr, brig...","[3438999494, 2882350436, 1701582961, 266817132...",[RT @BellDavidC: #HillaryPickUpLines Are you a...,"[2016-04-18 13:11, 2016-08-04 14:04, 2015-05-2...","[404468324, 404468324, 404468324, 404468324, 4...",632
101,ElPenguinito,460258316,"[LughHalley, JeanetteDBolden, WorldOfHashtags,...","[2614933193, 1670839033, 3438999494, 168340807...",[RT @ElPenguinito: #POTUSLastTweet I'm not tel...,"[2017-01-18 16:46, 2016-12-26 14:41, 2016-12-1...","[460258316, 460258316, 460258316, 460258316, 4...",331
102,hashtagnow_co,706424410450882560,"[LughHalley, GiselleEvns, traceyhappymom, Worl...","[2614933193, 2882013788, 2531159968, 343899949...",[RT @hashtagnow_co: #MustBeBanned #WorstPainEv...,"[2016-08-24 18:27, 2016-08-24 14:53, 2016-08-2...","[706424410450882560, 706424410450882560, 70642...",25
103,egorowww,2212973074,"[babchenkova, babchenkova, babchenkova, babche...","[383927282, 383927282, 383927282, 383927282, 3...",[RT @ivkudriavin: @azakuskin @egorowww В Литве...,"[2015-12-27 08:46, 2015-12-27 14:54, 2016-04-2...","[277075275, 159071440, 2212973074, 2212973074,...",146
104,WayWEcIT,3697635196,"[CovfefeNationUS, CovfefeNationUS, CynthiaMHun...","[743166519157227520, 743166519157227520, 16516...",[@WayWEcIT @GeorgiaDirtRoad @JrcheneyJohn http...,"[2017-11-18 21:10, 2017-11-18 21:10, 2016-11-1...","[3697635196, 806579094729224192, 975217231, 36...",48
105,DntTlkRae2Death,146999764,"[BleepThePolice, gloed_up]","[1687183549, 3312143142]",[RT @DntTlkRae2Death: WOW https://t.co/7lRE7nV...,"[2016-10-26 19:42, 2015-11-02 09:43]","[146999764, 146999764]",2
106,derrold,77321664,"[DanaGeezus, traceyhappymom, JeaNuttella, Aman...","[2882037326, 2531159968, 1655815544, 261634466...",[RT @derrold: #MyMondayDemotivation I'm workin...,"[2016-03-28 14:25, 2016-11-16 15:49, 2015-08-1...","[77321664, 77321664, 928249712, 2323982664, 21...",134
107,St0neC0ldSt0ner,102089838,"[RobertEbonyKing, ExQuote, ExQuote, ExQuote]","[1691000604, 3272640600, 3272640600, 3272640600]",[RT @St0neC0ldSt0ner: Me everyday. I need a pe...,"[2017-01-20 20:13, 2015-08-28 02:08, 2015-08-2...","[102089838, 102089838, 499463053, 457155272, 2...",15
108,Under6ftNoBeard,32204450,"[WatchMeWalkin, aantiracist, BigBoySneed, Jery...","[1688370956, 1709314117, 1868810166, 159488741...",[RT @Under6ftNoBeard: What if Futures days are...,"[2016-01-04 19:58, 2016-04-07 18:04, 2016-01-0...","[32204450, 32204450, 32204450, 274150578, 2726...",230
109,grosby81,2501427505,"[EricaRutter, AndyHashtagger, GiselleEvns, Dai...","[2540611879, 2878166306, 2882013788, 165769451...",[RT @grosby81: #ThingsThatComeFromCali White ...,"[2017-01-14 09:36, 2016-03-07 14:25, 2016-11-1...","[2501427505, 2501427505, 2501427505, 250142750...",367


In [78]:
# print(temp)

In [84]:
newContactDF.to_csv('contact_IRAtoUser.csv')